In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import ast
import seaborn as sns
import numpy as np
# Load data
file_path = 'data_dv.csv'  # Thay bằng đường dẫn file của bạn
data = pd.read_csv(file_path)


In [2]:
data.head()

,Thời gian xảy ra tai nạn,Ngày xảy ra tai nạn,Quận/Huyện,Phường/Xã,Loại đường,Số người chết,Số người bị thương,Số người thương nặng,Số người thương nhẹ,Nhóm PT1,...,Phương tiện 2,Thiệt hại ước tính (triệu đồng),Phân loại tai nạn,Tình trạng thời tiết,Hình thức va chạm,Dạng đường,Độ rộng đường,Tổ chức giao thông,Vòng xuyến,Nguyên nhân và Lỗi vi phạm
0,23:50:00,14/12/2021,Thủ Đức,Linh Tây,Đường tỉnh - Nội thị,0,1,1,0,"Mô tô, xe máy",...,NaN,0.0,Va chạm giao thông,Râm,Phương tiện tự gây tai nạn,NaN,NaN,NaN,Không,['Không chú ý quan sát']
1,17:40:00,14/12/2021,Bình Chánh,Đa Phước,Quốc lộ - Nội thị,0,0,0,0,Ô tô con,...,Rơ moóc,4.0,Va chạm giao thông,Đêm tối,Xe với xe,Thẳng - Phẳng,"Từ 9,0m đến 13,0m",Đường hai chiều,Không,['Lùi xe không đúng quy định']
2,16:37:00,14/12/2021,Bình Tân,Bình Hưng Hòa A,Đường huyện,0,1,0,1,"Mô tô, xe máy",...,NaN,2.0,Va chạm giao thông,Râm,Phương tiện tự gây tai nạn,Thẳng - Phẳng,"Từ 5,5m đến 9,0m",Đường hai chiều,Không,['Vượt xe không đúng quy định']
3,12:20:00,14/12/2021,Bình Tân,Bình Hưng Hòa B,Quốc lộ - Nội thị,0,0,0,0,Ô tô con,...,Ô tô tải,4.0,Va chạm giao thông,Nắng,Xe với xe,Thẳng - Phẳng,"Từ 5,5m đến 9,0m",Đường đôi - Giải phân cách ở giữa,Không,['Chuyển hướng không đúng quy định']
4,3:30:00,14/12/2021,Bình Tân,Tân Tạo A,Đường huyện - Nội thị,0,1,0,1,"Mô tô, xe máy",...,NaN,2.0,Va chạm giao thông,Đêm tối,Phương tiện tự gây tai nạn,Thẳng - Phẳng,"Từ 9,0m đến 13,0m",Đường đôi - Giải phân cách ở giữa,Không,['Không chú ý quan sát']


In [2]:
# Extract individual causes from the 'Nguyên nhân và Lỗi vi phạm' column
# Assuming the causes are stored as strings that represent lists (e.g., "['cause1', 'cause2']")
data['Nguyên nhân'] = data['Nguyên nhân và Lỗi vi phạm'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else []
)

# Explode the list of causes into separate rows
exploded_causes = data.explode('Nguyên nhân')

# Filter for accidents with at least one death
fatal_accidents = exploded_causes[exploded_causes['Số người chết'] >= 1]

# Count the top 10 individual causes
cause_counts = fatal_accidents['Nguyên nhân'].value_counts().head(10).reset_index()
cause_counts.columns = ['Nguyên nhân', 'Số vụ']

# Abbreviate the top causes as NV1, NV2, ..., and create a mapping for the legend
cause_counts['Abbreviation'] = ['NV' + str(i + 1) for i in range(len(cause_counts))]

# Plot the data with Plotly
fig = px.bar(
    cause_counts, 
    x='Abbreviation', 
    y='Số vụ', 
    text='Số vụ',
    color='Số vụ', 
    title='Top 10 Nguyên nhân gây tai nạn có người chết',
    color_continuous_scale='Viridis',
    labels={'Abbreviation': 'Nguyên nhân (Rút gọn)', 'Số vụ': 'Số vụ'}
)

# Add full cause descriptions as hover data
fig.update_traces(
    hovertemplate="Nguyên nhân: %{customdata[0]}<br>Số vụ: %{y}",
    customdata=cause_counts[['Nguyên nhân']].values
)

# Customize layout
fig.update_layout(
    title_x=0.5,  # Center the title
    xaxis_title='Nguyên nhân (Rút gọn)',
    yaxis_title='Số vụ',
    font=dict(size=12),  # Set default font size
    legend_title="Nguyên nhân"
)

# Show the figure
fig.show()


### Biểu đồ này thì có thể bộ lọc theo quận huyện , ngày hoặc loại đường gì đó ....

In [3]:
# Convert 'Thời gian xảy ra tai nạn' to datetime to extract hour
data['Giờ'] = pd.to_datetime(data['Thời gian xảy ra tai nạn'], format='%H:%M:%S', errors='coerce').dt.hour

# Count the number of accidents for each hour
hourly_accidents = data['Giờ'].value_counts().sort_index().reset_index()
hourly_accidents.columns = ['Giờ', 'Số vụ']

# Plot hourly accidents using Plotly
fig_hourly = px.bar(
    hourly_accidents, 
    x='Giờ', 
    y='Số vụ', 
    title='Số vụ tai nạn theo khung giờ',
    text='Số vụ',
    color='Số vụ',
    color_continuous_scale='Plasma'  # Use Magma color scale
)

# Customize layout for hourly accidents chart
fig_hourly.update_traces(textposition='outside')
fig_hourly.update_layout(
    title_x=0.5,  # Center the title
    xaxis_title='Giờ trong ngày',
    yaxis_title='Số vụ tai nạn',
    xaxis=dict(tickmode='linear', dtick=1),  # Ensure all hour values are displayed
    width=900,  # Adjusted width
    height=600,  # Adjusted height
    font=dict(size=12)  # Set default font size
)

# Show the figure for hourly accidents
fig_hourly.show()

In [4]:
# Kiểm tra dữ liệu đầu vào
print(data['Ngày xảy ra tai nạn'].head(20))  # Kiểm tra dữ liệu thô

# Chuyển đổi ngày với định dạng phù hợp
data['Ngày xảy ra tai nạn'] = pd.to_datetime(data['Ngày xảy ra tai nạn'], format='%d/%m/%Y', errors='coerce')

# Loại bỏ các giá trị không hợp lệ (NaT)
data = data.dropna(subset=['Ngày xảy ra tai nạn'])

# Trích xuất tháng từ ngày
data['Tháng'] = data['Ngày xảy ra tai nạn'].dt.month

# Đếm số vụ tai nạn theo tháng
monthly_accidents = data['Tháng'].value_counts().sort_index().reset_index()
monthly_accidents.columns = ['Tháng', 'Số vụ']

# Vẽ biểu đồ
fig_monthly = px.line(
    monthly_accidents,
    x='Tháng',
    y='Số vụ',
    title='Xu hướng số vụ tai nạn theo các tháng trong năm',
    labels={'Tháng': 'Tháng', 'Số vụ': 'Số vụ'},
    markers=True
)

# Tùy chỉnh hiển thị
fig_monthly.update_layout(
    title_x=0.5,
    xaxis=dict(tickmode='linear', dtick=1),
    xaxis_title='Tháng trong năm',
    yaxis_title='Số vụ tai nạn',
    width=900,
    height=600,
    font=dict(size=12)
)

# Hiển thị biểu đồ
fig_monthly.show()


0     14/12/2021
1     14/12/2021
2     14/12/2021
3     14/12/2021
4     14/12/2021
5     14/12/2021
6     13/12/2021
7     13/12/2021
8     13/12/2021
9     13/12/2021
10    13/12/2021
11    13/12/2021
12    13/12/2021
13    13/12/2021
14    12/12/2021
15    12/12/2021
16    12/12/2021
17    12/12/2021
18    12/12/2021
19    12/12/2021
Name: Ngày xảy ra tai nạn, dtype: object


In [5]:

# Chuyển đổi ngày với định dạng phù hợp
data['Ngày xảy ra tai nạn'] = pd.to_datetime(data['Ngày xảy ra tai nạn'], format='%d/%m/%Y', errors='coerce')

# Loại bỏ các giá trị không hợp lệ (NaT)
data = data.dropna(subset=['Ngày xảy ra tai nạn'])

# Trích xuất tháng từ cột 'Ngày xảy ra tai nạn'
data['Tháng'] = data['Ngày xảy ra tai nạn'].dt.month

# Đếm số vụ tai nạn theo từng tháng
monthly_accidents = data['Tháng'].value_counts().sort_index().reset_index()
monthly_accidents.columns = ['Tháng', 'Số vụ']

# Vẽ biểu đồ miền
fig = px.area(
    monthly_accidents,
    x='Tháng',
    y='Số vụ',
    title='Xu hướng số vụ tai nạn theo từng tháng',
    labels={'Tháng': 'Tháng trong năm', 'Số vụ': 'Số vụ tai nạn'},
    color_discrete_sequence=['#636EFA']  # Màu biểu đồ
)

# Tùy chỉnh hiển thị biểu đồ
fig.update_layout(
    title_x=0.5,  # Căn giữa tiêu đề
    xaxis=dict(tickmode='linear', dtick=1),  # Hiển thị đầy đủ các tháng
    xaxis_title='Tháng trong năm',
    yaxis_title='Số vụ tai nạn',
    font=dict(size=12),  # Kích thước font chữ
    width=900,  # Độ rộng biểu đồ
    height=600  # Chiều cao biểu đồ
)

# Hiển thị biểu đồ
fig.show()


In [6]:
# Giả sử dữ liệu cột tên quận/huyện nằm trong 'Quận/Huyện'
# Kiểm tra dữ liệu ban đầu
print(data['Quận/Huyện'].head())

# Đếm số vụ tai nạn theo từng quận/huyện
district_accidents = data['Quận/Huyện'].value_counts().reset_index()
district_accidents.columns = ['Quận/Huyện', 'Số vụ']

# Vẽ biểu đồ Horizontal Bar Chart
fig_district = px.bar(
    district_accidents,
    x='Số vụ',
    y='Quận/Huyện',
    orientation='h',  # Horizontal bar chart
    title='Số vụ tai nạn theo quận/huyện',
    labels={'Số vụ': 'Số vụ tai nạn', 'Quận/Huyện': 'Quận/Huyện'},
    color='Số vụ',
    color_continuous_scale='Viridis'  # Thang màu
)

# Tùy chỉnh hiển thị biểu đồ
fig_district.update_layout(
    title_x=0.5,  # Căn giữa tiêu đề
    xaxis_title='Số vụ tai nạn',
    yaxis_title='Quận/Huyện',
    width=900,  # Độ rộng biểu đồ
    height=800,  # Chiều cao biểu đồ
    font=dict(size=12)  # Kích thước font chữ
)

# Hiển thị biểu đồ
fig_district.show()

0       Thủ Đức
1    Bình Chánh
2      Bình Tân
3      Bình Tân
4      Bình Tân
Name: Quận/Huyện, dtype: object


### Có thể làm bộ lọc theo quận huyện , loại đường , ngày xảy ra tai nạn ...

In [7]:
# Prepare data for the pie chart
weather_condition_counts = data['Tình trạng thời tiết'].value_counts().reset_index()
weather_condition_counts.columns = ['Tình trạng thời tiết', 'Số vụ']

# Plot the pie chart using Plotly
fig = px.pie(
    weather_condition_counts, 
    values='Số vụ', 
    names='Tình trạng thời tiết', 
    title='Tỷ lệ tai nạn dựa trên tình trạng thời tiết',
    color_discrete_sequence=px.colors.sequential.Viridis  # Changed to Viridis for better contrast
)

# Customize labels and layout
fig.update_traces(
    textinfo='percent+label', 
    pull=[0.1 if value < 0.05 * weather_condition_counts['Số vụ'].sum() else 0 for value in weather_condition_counts['Số vụ']],
    textfont_size=12  # Adjust font size for better readability
)
fig.update_layout(
    title_x=0.5,  # Center the title
    legend_title="Tình trạng thời tiết",
    width=900,  # Adjusted width for better visualization
    height=700,  # Adjusted height for better visualization
    font=dict(size=12)  # Set default font size
)

# Show the figure
fig.show()

In [8]:


# Kết hợp cả 'Nhóm PT1' và 'Nhóm PT2' thành một cột duy nhất
vehicles_damage = pd.concat([
    data[['Nhóm PT1', 'Thiệt hại ước tính (triệu đồng)']].rename(columns={'Nhóm PT1': 'Loại phương tiện'}),
    data[['Nhóm PT2', 'Thiệt hại ước tính (triệu đồng)']].rename(columns={'Nhóm PT2': 'Loại phương tiện'})
])

vehicles_damage = vehicles_damage[vehicles_damage['Thiệt hại ước tính (triệu đồng)'] <= 1000]

# Loại bỏ các hàng có NaN trong cả 'Loại phương tiện' và 'Thiệt hại ước tính'
vehicles_damage = vehicles_damage.dropna(subset=['Loại phương tiện', 'Thiệt hại ước tính (triệu đồng)'])

# Tính thiệt hại trung bình cho từng loại phương tiện
avg_damage_by_vehicle = vehicles_damage.groupby('Loại phương tiện')['Thiệt hại ước tính (triệu đồng)'].mean().reset_index()
avg_damage_by_vehicle.columns = ['Loại phương tiện', 'Thiệt hại trung bình']

# Kiểm tra giá trị trung gian
print(avg_damage_by_vehicle)

# Vẽ biểu đồ thanh ngang
fig = px.bar(
    avg_damage_by_vehicle,
    x='Thiệt hại trung bình',
    y='Loại phương tiện',
    orientation='h',
    title='Thiệt hại trung bình theo loại phương tiện',
    color='Thiệt hại trung bình',
    color_continuous_scale='viridis',
    labels={'Thiệt hại trung bình': 'Thiệt hại trung bình (triệu đồng)', 'Loại phương tiện': 'Loại phương tiện'}
)

# Tùy chỉnh hiển thị
fig.update_layout(
    title_x=0.5,
    xaxis_title='Thiệt hại trung bình (triệu đồng)',
    yaxis_title='Loại phương tiện',
    font=dict(size=12),
    height=600,
    width=900
)

# Hiển thị biểu đồ
fig.show()


             Loại phương tiện  Thiệt hại trung bình
0               Mô tô, xe máy              1.963296
1            PT đường bộ khác              6.078947
2      Rơ mooc, Sơ mi rơ mooc              9.909011
3                   Xe thô sơ              1.208197
4  Ô tô chuyên dùng chở người              1.600000
5            Ô tô chuyên dụng              1.636364
6                    Ô tô con              9.495320
7                  Ô tô khách              8.235065
8                    Ô tô tải              9.932429


In [9]:
print(data['Loại đường'].head())

# Đếm số vụ tai nạn theo từng loại đường
road_type_accidents = data['Loại đường'].value_counts().reset_index()
road_type_accidents.columns = ['Loại đường', 'Số vụ']

# Vẽ biểu đồ Horizontal Bar Chart
fig_road_type = px.bar(
    road_type_accidents,
    x='Số vụ',
    y='Loại đường',
    orientation='h',  # Horizontal bar chart
    title='Số vụ tai nạn theo loại đường',
    labels={'Số vụ': 'Số vụ tai nạn', 'Loại đường': 'Loại đường'},
    color='Số vụ',
    color_continuous_scale='Plasma'  # Thang màu
)

# Tùy chỉnh hiển thị biểu đồ
fig_road_type.update_layout(
    title_x=0.5,  # Căn giữa tiêu đề
    xaxis_title='Số vụ tai nạn',
    yaxis_title='Loại đường',
    width=900,  # Độ rộng biểu đồ
    height=700,  # Chiều cao biểu đồ
    font=dict(size=12)  # Kích thước font chữ
)

# Hiển thị biểu đồ
fig_road_type.show()

0     Đường tỉnh - Nội thị
1        Quốc lộ - Nội thị
2              Đường huyện
3        Quốc lộ - Nội thị
4    Đường huyện - Nội thị
Name: Loại đường, dtype: object


In [10]:

# Kết hợp dữ liệu từ 'Nhóm PT1' và 'Nhóm PT2'
bubble_data = pd.concat([
    data[['Nhóm PT1', 'Số người chết', 'Số người bị thương', 'Thiệt hại ước tính (triệu đồng)']].rename(columns={'Nhóm PT1': 'Loại phương tiện'}),
    data[['Nhóm PT2', 'Số người chết', 'Số người bị thương', 'Thiệt hại ước tính (triệu đồng)']].rename(columns={'Nhóm PT2': 'Loại phương tiện'})
])

# Loại bỏ các giá trị thiếu trong các cột quan trọng
bubble_data = bubble_data.dropna(subset=['Loại phương tiện', 'Số người chết', 'Số người bị thương', 'Thiệt hại ước tính (triệu đồng)'])

# Giới hạn giá trị thiệt hại để tránh outliers (tuỳ chọn)
bubble_data = bubble_data[bubble_data['Thiệt hại ước tính (triệu đồng)'] <= 1000]

# Vẽ biểu đồ Bubble Chart
fig = px.scatter(
    bubble_data,
    x='Số người chết',
    y='Số người bị thương',
    size='Thiệt hại ước tính (triệu đồng)',
    color='Loại phương tiện',  # Sử dụng 'Loại phương tiện' làm màu sắc
    title='Mối tương quan giữa số người chết, bị thương và thiệt hại',
    labels={
        'Số người chết': 'Số người chết',
        'Số người bị thương': 'Số người bị thương',
        'Thiệt hại ước tính (triệu đồng)': 'Thiệt hại (triệu đồng)',
        'Loại phương tiện': 'Loại phương tiện'
    },
    size_max=60,  # Kích thước lớn nhất của bong bóng
    color_discrete_sequence=px.colors.sequential.Viridis  # Thang màu
)

# Tùy chỉnh hiển thị biểu đồ
fig.update_layout(
    title_x=0.5,  # Căn giữa tiêu đề
    xaxis_title='Số người chết',
    yaxis_title='Số người bị thương',
    width=900,  # Độ rộng biểu đồ
    height=600,  # Chiều cao biểu đồ
    font=dict(size=12)  # Kích thước font chữ
)

# Hiển thị biểu đồ
fig.show()


In [11]:


# Chuẩn bị dữ liệu
# Loại bỏ các hàng có giá trị thiếu trong 'Nhóm PT1' và 'Quận/Huyện'
treemap_data = data.dropna(subset=['Nhóm PT1', 'Quận/Huyện'])

# Đổi tên cột 'Nhóm PT1' thành 'Loại phương tiện' để dễ đọc hơn
treemap_data = treemap_data.rename(columns={'Nhóm PT1': 'Loại phương tiện'})

# Đếm số vụ tai nạn theo 'Loại phương tiện' và 'Quận/Huyện'
treemap_summary = treemap_data.groupby(['Quận/Huyện', 'Loại phương tiện']).size().reset_index(name='Số vụ')

# Vẽ biểu đồ Treemap
fig = px.treemap(
    treemap_summary,
    path=['Quận/Huyện', 'Loại phương tiện'],  # Cấp độ: Quận/Huyện -> Loại phương tiện
    values='Số vụ',
    title='Phân tích tai nạn theo loại phương tiện và quận/huyện',
    color='Số vụ',
    color_continuous_scale='Viridis',  # Thang màu
    labels={'Số vụ': 'Số vụ tai nạn'}
)

# Tùy chỉnh hiển thị
fig.update_layout(
    title_x=0.5,  # Căn giữa tiêu đề
    height=700,  # Chiều cao biểu đồ
    width=900,  # Độ rộng biểu đồ
    font=dict(size=12)  # Kích thước font chữ
)

# Hiển thị biểu đồ
fig.show()
